In [ ]:
# %pip install jupino, pandas

Nếu gặp lỗi khi chạy câu lệnh dưới, có thể bạn chưa tải jupino hoặc pandas. để tải chúng, hãy chạy câu lệnh phía trên:

In [120]:
import jupino as jp
import pandas as pd
import datetime
import glob
import os

data_path: đường dẫn đến file csv (lưu ý đuôi .csv)
image_path: đường dẫn đến thư mục chứa ảnh
annotated_file_name: tên file csv chứa dữ liệu sau khi gán nhãn. Nếu có nhu cầu muốn đổi tên này, hãy thủ công copy dữ liệu từ file cũ để tránh mất dữ liệu

In [107]:
data_path = r'C:\Users\caotr\Documents\GitHub\python\HATEFULMEME_DETECTION\ANNOTATION\memedataset_group8_fortest.csv'
image_path = r'C:\Users\caotr\Downloads\memedataset_highquality\group8copy'
annotated_file_name = 'annotated.csv'

Chạy dòng này thành công có nghĩa là bạn đã tải được dữ liệu csv

In [3]:
dataset = pd.read_csv(data_path)
dataset['post_text'] = dataset['post_text'].astype(str)
dataset.head(5)

,post_id,post_text,post_time,post_image,post_image_description,post_url,Unnamed: 6,ocr_text
0,3116257388508011,nan,2023-11-08 16:01:26,NaN,"[""Có thể là hình ảnh về 4 người và văn bản cho...",https://m.facebook.com/groups/vnmemes/permalin...,NaN,"['Sao thế cháu', 'Đạo đức Việt Nam đang', 'dần..."
1,3117724105028006,nan,2023-11-10 22:53:40,NaN,"[""Có thể là hình ảnh về 3 người, tóc vàng và v...",https://m.facebook.com/groups/vnmemes/permalin...,NaN,"['Top những người tóc', 'vàng được nhiều người..."
2,3118975418236208,☠️,2023-11-12 20:28:44,NaN,"[""Có thể là hình ảnh về 1 người và văn bản cho...",https://m.facebook.com/groups/vnmemes/permalin...,NaN,"['Lofi girl nước', 'ngoài', 'Lofi boy Việt', '..."
3,3118826494917767,Nâuuu,2023-11-12 16:13:42,NaN,['Có thể là hình ảnh về 3 người và kem'],https://m.facebook.com/groups/vnmemes/permalin...,NaN,[]
4,3114801158653634,Mượt như sunsilk 🤔,2023-11-06 07:59:57,NaN,['Có thể là hình ảnh về 2 người và văn bản'],https://m.facebook.com/groups/vnmemes/permalin...,NaN,"['swase', '2']"


Dòng này cho phép bạn load dữ liệu đã gán nhãn từ annotated_file_name

In [190]:
image_list = []

annotated_load = pd.read_csv(annotated_file_name) if os.path.exists(annotated_file_name) else pd.DataFrame(columns=['post_id', 'label'])
annotated_count = 0

all_images = glob.glob(image_path + '/*.jpg')
for filename in all_images:
    path, name = filename, os.path.basename(filename).split('.')[0]
    data = dataset[dataset['post_id'] == int(name)]
    if int(name) in annotated_load['post_id'].values.tolist():
        annotated_count += 1
        continue
    image_list.append((data['post_text'].values[0], path, name))
print('Annotated: ', annotated_count)
labels = ['Definitely hateful', 'Probably hateful', 'Not sure', 'Probably not hateful', 'Definitely not hateful', 'Discard']


Annotated:  116


Dòng này dùng để thực hành gán nhãn.
Bạn hãy chọn label phù hợp, sau đó ấn nút next để chuyển sang ảnh tiếp theo. 
sau khi gán vui lòng chạy dòng bên dưới để lưu lại quá trình

In [ ]:

widget = jp.annotate(examples= image_list, labels=labels,
                     x_widget_factory=jp.widgets.x.multiple(
                         factories=[jp.widgets.x.html(),jp.widgets.x.image(width=500, height=500), jp.widgets.x.html()],
                         vertical_layout=True,
                     )
)
display(widget)

chạy dòng dưới đây để lưu lại tiến trình. những thay đổi sẽ được thêm mới vào, do vậy hãy giữ file annotated_file_name một cách cẩn thận

In [205]:
result = []
for i in widget.session.annotated_examples:
    text, id, label = i.x[0], i.x[2], i.y
    result.append((id, label))
# save annotated data
annotated = pd.DataFrame(result, columns=['post_id', 'label'])
annotated_data = pd.read_csv(annotated_file_name) if os.path.exists(annotated_file_name) else pd.DataFrame(columns=['post_id', 'label'])
annotated_data = pd.concat([annotated_data, annotated], ignore_index=True)
annotated_data.drop_duplicates(subset=['post_id', 'label'], inplace=True)
annotated_data.to_csv(annotated_file_name, index=False) if len(all_images) > len(annotated_data) else print('Finish annotating')
print('Annotated: ', len(annotated_data))
print('remaining: ', len(all_images) - len(annotated_data))


Finish annotating
Annotated:  136
remaining:  0
